# 使用Sequential API构建图像分类MLP

本教程演示如何使用Keras的Sequential API构建多层感知机(MLP)来解决图像分类问题。

## 学习目标

1. 理解图像分类任务的数据预处理流程
2. 掌握多分类MLP的模型架构设计
3. 学会使用交叉熵损失函数
4. 理解Softmax输出和概率预测

## 数据集介绍

Fashion-MNIST数据集包含10类时尚商品的灰度图像：
- 60,000张训练图像
- 10,000张测试图像
- 图像尺寸：28×28像素
- 10个类别：T恤、裤子、套头衫、连衣裙、外套、凉鞋、衬衫、运动鞋、包、短靴

## 1. 环境配置

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

# 设置随机种子确保结果可复现
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

print(f"TensorFlow版本: {tf.__version__}")

## 2. 数据加载与探索

In [ ]:
# 使用Keras内置API加载Fashion-MNIST数据集
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

# 查看数据集基本信息
print("数据集形状:")
print(f"训练集: {X_train_full.shape}")
print(f"测试集: {X_test.shape}")
print(f"\n数据类型: {X_train_full.dtype}")
print(f"像素值范围: [{X_train_full.min()}, {X_train_full.max()}]")

In [ ]:
# 定义类别名称
class_names = [
    'T-shirt/top',  # T恤
    'Trouser',      # 裤子
    'Pullover',     # 套头衫
    'Dress',        # 连衣裙
    'Coat',         # 外套
    'Sandal',       # 凉鞋
    'Shirt',        # 衬衫
    'Sneaker',      # 运动鞋
    'Bag',          # 包
    'Ankle boot'    # 短靴
]

# 查看标签分布
print("训练集标签分布:")
unique, counts = np.unique(y_train_full, return_counts=True)
for label, count in zip(unique, counts):
    print(f"  {class_names[label]}: {count}")

In [ ]:
# 可视化部分样本
def plot_samples(X, y, class_names, n_rows=3, n_cols=5):
    """
    绘制数据集样本
    
    Parameters:
    -----------
    X : ndarray
        图像数据
    y : ndarray
        标签数据
    class_names : list
        类别名称列表
    n_rows, n_cols : int
        行数和列数
    """
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 7))
    for i, ax in enumerate(axes.flat):
        ax.imshow(X[i], cmap='gray')
        ax.set_title(class_names[y[i]], fontsize=10)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

plot_samples(X_train_full, y_train_full, class_names)

## 3. 数据预处理

### 关键步骤

1. **划分验证集**: 从训练集中分离出验证数据
2. **归一化**: 将像素值从[0, 255]缩放到[0, 1]
3. **展平**: MLP需要一维输入（Flatten层会在模型中处理）

In [ ]:
# 划分验证集（前5000个样本作为验证集）
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

print(f"训练集大小: {X_train.shape[0]}")
print(f"验证集大小: {X_valid.shape[0]}")
print(f"测试集大小: {X_test.shape[0]}")

In [ ]:
# 归一化：将像素值缩放到[0, 1]范围
# 这对梯度下降优化至关重要
X_train = X_train / 255.0
X_valid = X_valid / 255.0
X_test = X_test / 255.0  # 测试集也必须归一化！

print(f"归一化后像素值范围: [{X_train.min():.1f}, {X_train.max():.1f}]")

## 4. 构建MLP分类模型

### 模型架构

```
输入层 (28×28 = 784个像素)
    ↓ Flatten
展平层 (784个神经元)
    ↓
隐藏层1 (300个神经元, ReLU)
    ↓
隐藏层2 (100个神经元, ReLU)
    ↓
输出层 (10个神经元, Softmax)
```

### 设计要点

- **Flatten层**: 将2D图像展平为1D向量
- **ReLU激活**: 隐藏层使用ReLU解决梯度消失问题
- **Softmax激活**: 输出层使用Softmax生成概率分布

In [ ]:
# 构建Sequential模型
model = keras.Sequential([
    # 输入层：Flatten将28×28图像展平为784维向量
    keras.layers.Flatten(input_shape=[28, 28]),
    
    # 隐藏层1：300个神经元，ReLU激活
    keras.layers.Dense(300, activation='relu'),
    
    # 隐藏层2：100个神经元，ReLU激活
    keras.layers.Dense(100, activation='relu'),
    
    # 输出层：10个神经元（对应10个类别），Softmax激活
    keras.layers.Dense(10, activation='softmax')
])

# 查看模型结构
model.summary()

In [ ]:
# 查看模型参数详情
print("模型层信息:")
print("="*60)
for i, layer in enumerate(model.layers):
    print(f"层{i}: {layer.name}")
    if hasattr(layer, 'kernel'):
        weights, biases = layer.get_weights()
        print(f"  权重形状: {weights.shape}")
        print(f"  偏置形状: {biases.shape}")
        print(f"  参数总数: {weights.size + biases.size}")

## 5. 编译模型

### 损失函数选择

- **sparse_categorical_crossentropy**: 标签为整数形式 (0, 1, 2, ...)
- **categorical_crossentropy**: 标签为one-hot编码形式

我们的标签是整数形式，所以使用sparse_categorical_crossentropy。

In [ ]:
model.compile(
    loss='sparse_categorical_crossentropy',  # 多分类交叉熵损失
    optimizer='sgd',                          # SGD优化器
    metrics=['accuracy']                      # 监控准确率
)

# 等效的显式写法
# model.compile(
#     loss=keras.losses.SparseCategoricalCrossentropy(),
#     optimizer=keras.optimizers.SGD(learning_rate=0.01),
#     metrics=[keras.metrics.SparseCategoricalAccuracy()]
# )

## 6. 训练模型

In [ ]:
# 训练模型
history = model.fit(
    X_train, y_train,
    epochs=30,                              # 训练30个epoch
    batch_size=32,                          # 批量大小
    validation_data=(X_valid, y_valid),     # 验证数据
    verbose=1
)

## 7. 可视化训练过程

In [ ]:
# 绘制学习曲线
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 损失曲线
axes[0].plot(history.history['loss'], label='Training Loss')
axes[0].plot(history.history['val_loss'], label='Validation Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Loss Curves')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 准确率曲线
axes[1].plot(history.history['accuracy'], label='Training Accuracy')
axes[1].plot(history.history['val_accuracy'], label='Validation Accuracy')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Accuracy Curves')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 使用pandas绘制所有指标
history_df = pd.DataFrame(history.history)
history_df.plot(figsize=(10, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.xlabel('Epoch')
plt.ylabel('Metric Value')
plt.title('Training History')
plt.show()

## 8. 模型评估

In [ ]:
# 在测试集上评估模型
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

print("测试集评估结果:")
print(f"损失 (Cross-Entropy): {test_loss:.4f}")
print(f"准确率: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

## 9. 使用模型进行预测

In [ ]:
# 对测试集前3个样本进行预测
X_new = X_test[:3]
y_true = y_test[:3]

# predict()返回每个类别的概率
y_proba = model.predict(X_new, verbose=0)

print("预测概率分布:")
print(y_proba.round(3))

In [ ]:
# 获取预测类别（概率最高的类别）
y_pred = np.argmax(y_proba, axis=1)

print("预测结果:")
print("="*50)
for i in range(len(X_new)):
    true_label = class_names[y_true[i]]
    pred_label = class_names[y_pred[i]]
    confidence = y_proba[i][y_pred[i]] * 100
    status = "Correct" if y_true[i] == y_pred[i] else "Wrong"
    print(f"样本{i+1}: 真实={true_label}, 预测={pred_label}, 置信度={confidence:.1f}% [{status}]")

In [ ]:
# 可视化预测结果
def plot_prediction(X, y_true, y_proba, class_names, n_samples=5):
    """
    可视化模型预测结果
    """
    fig, axes = plt.subplots(2, n_samples, figsize=(12, 5))
    
    for i in range(n_samples):
        # 显示图像
        axes[0, i].imshow(X[i], cmap='gray')
        pred_label = np.argmax(y_proba[i])
        color = 'green' if y_true[i] == pred_label else 'red'
        axes[0, i].set_title(f"True: {class_names[y_true[i]]}\n"
                             f"Pred: {class_names[pred_label]}", 
                             color=color, fontsize=9)
        axes[0, i].axis('off')
        
        # 显示概率分布
        axes[1, i].barh(range(10), y_proba[i])
        axes[1, i].set_yticks(range(10))
        axes[1, i].set_yticklabels(class_names, fontsize=7)
        axes[1, i].set_xlim(0, 1)
        axes[1, i].axvline(x=0.5, color='gray', linestyle='--', alpha=0.5)
    
    plt.tight_layout()
    plt.show()

# 可视化前5个测试样本的预测
X_sample = X_test[:5]
y_sample = y_test[:5]
y_proba_sample = model.predict(X_sample, verbose=0)
plot_prediction(X_sample, y_sample, y_proba_sample, class_names)

## 10. 混淆矩阵分析

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# 对整个测试集进行预测
y_pred_all = np.argmax(model.predict(X_test, verbose=0), axis=1)

# 计算混淆矩阵
cm = confusion_matrix(y_test, y_pred_all)

# 可视化混淆矩阵
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()

In [ ]:
# 打印分类报告
print("分类报告:")
print("="*60)
print(classification_report(y_test, y_pred_all, target_names=class_names))

## 小结

### 多分类任务的关键点

1. **数据归一化**: 像素值缩放到[0,1]对训练至关重要
2. **输出层设计**: 使用Softmax激活，神经元数等于类别数
3. **损失函数**: 使用交叉熵损失（sparse或one-hot编码）
4. **预测输出**: Softmax输出概率分布，使用argmax获取类别

### 模型性能分析

从混淆矩阵可以看出：
- 容易混淆的类别：Shirt vs T-shirt/top, Pullover vs Coat
- 这些类别在视觉上确实相似

### 改进方向

1. 增加网络深度或宽度
2. 添加Dropout正则化
3. 使用更高级的优化器(Adam)
4. 使用数据增强
5. 使用卷积神经网络(CNN)替代MLP